<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/DL_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import os
import zipfile

In [7]:
extract_dir = "/content/Qualcomm-DL-Hackathon/Images"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("/content/Qualcomm-DL-Hackathon/train/images_part-1.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/Qualcomm-DL-Hackathon/Images
